### Manually Download Code

In [1]:
# Required Libraries, do not touch
import sys
import os
from configparser import ConfigParser
from datetime import datetime, timedelta

# Add other libraries below
# import numpy
# import pandas

In [2]:
# Variables [Do not change]
manual = True

# Variables [You can change]
startdate = '2002-06-01' # For end date, get the current date, minus 1 day, in string format YYYY-MM-DD
enddate = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
sys.path.append('./main/')
from bmnp import BMNP_Data, BMNP_Download

data = BMNP_Data(startdate=startdate, enddate=enddate, manually=manual)
download = BMNP_Download(manually=manual)

[11:50:28] Dates Missing from SST Database: []


In [4]:
missing_dates = ['2024-07-07']

data.download.downloaded_dates = missing_dates
data.delete_singles = False
data.dates_missing = True
data.download.downloaded_dates

['2024-07-07']

In [5]:
data.createDHWs()

[11:50:28] Going through the else if data...
[11:50:28] Date Index: 8072
[11:50:28] New Lat: 30 New Lon: 24
[11:50:28] Min Lat: 12.020000457763672 Max Lat: 12.3100004196167
[11:50:28] Min Lon: -68.41999816894531 Max Lon: -68.19000244140625
[11:50:28] Date: 2024-07-07 Lat: 34 Lon: 27
[11:50:28] SST Data Shape: (0, 24, 27)
[11:50:28] Bleaching Threshold Shape: (30, 24)


ValueError: operands could not be broadcast together with shapes (0,24,27) (30,24) 

In [8]:
# Open first item in the "./data/nc_sst/" folder
first_item = os.listdir('./data/nc_sst/')[0]
first_item

'2021-10-23.nc'

In [9]:
# Load this using netcdf4
import netCDF4 as nc
nc_data = nc.Dataset('./data/nc_sst/' + first_item, 'r')
nc_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lon(27), lat(34), time(1)
    variables(dimensions): float32 lon(lon), float32 lat(lat), float32 time(time), float32 analysed_sst(time, lat, lon)
    groups: 

In [10]:
# Retrieve "lon" and "lat" variables
lons = nc_data.variables['lon'][:]
lats = nc_data.variables['lat'][:]

hrcs = nc.Dataset(f'./data/hrcs_mmm.nc', 'r')
bleaching_threshold = hrcs.variables['variable'][:] + 1.0

In [11]:
hrcs_lat = hrcs.variables['lat'][:]
hrcs_lon = hrcs.variables['lon'][:]
sst_lat = lats
sst_lon = lons
hrcs_lat = [round(lat, 2) for lat in hrcs_lat]
hrcs_lon = [round(lon, 2) for lon in hrcs_lon]
sst_lat = [round(lat, 2) for lat in sst_lat]
sst_lon = [round(lon, 2) for lon in sst_lon]

from numpy import sort
# Order in ascending order
hrcs_lat = sort(hrcs_lat)
hrcs_lon = sort(hrcs_lon)
sst_lat = sort(sst_lat)
sst_lon = sort(sst_lon)

# Print the first and last values of each.
print(f'First and last values of hrcs_lat: {hrcs_lat[0]}, {hrcs_lat[-1]}')
print(f'First and last values of sst_lat: {sst_lat[0]}, {sst_lat[-1]}')
print(f'First and last values of hrcs_lon: {hrcs_lon[0]}, {hrcs_lon[-1]}')
print(f'First and last values of sst_lon: {sst_lon[0]}, {sst_lon[-1]}')

First and last values of hrcs_lat: 12.020000457763672, 12.3100004196167
First and last values of sst_lat: 12.0, 12.329999923706055
First and last values of hrcs_lon: -68.41999816894531, -68.19000244140625
First and last values of sst_lon: -68.44000244140625, -68.18000030517578


In [12]:
# Give me the minimum and maximum in both lat and lon for the hrcs file
hrcs_min_lat = min(hrcs_lat)
hrcs_max_lat = max(hrcs_lat)
hrcs_min_lon = min(hrcs_lon)
hrcs_max_lon = max(hrcs_lon)

from numpy import where
# Based on the min and max lat and lon, find the indices in the sst file.
min_lat_idx = where(sst_lat == hrcs_min_lat)[0][0]
max_lat_idx = where(sst_lat == hrcs_max_lat)[0][0]+1
min_lon_idx = where(sst_lon == hrcs_min_lon)[0][0]
max_lon_idx = where(sst_lon == hrcs_max_lon)[0][0]+1

new_lat = sst_lat[min_lat_idx:max_lat_idx]
new_lon = sst_lon[min_lon_idx:max_lon_idx]

In [14]:
sst_data = nc_data.variables['analysed_sst'][0, min_lat_idx:max_lat_idx, min_lon_idx:max_lon_idx]
# Print shape
print(sst_data.shape)

(0, 24, 27)
